In [1]:
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_row", 160)
pd.set_option('max_colwidth', 200)

In [2]:
a = pd.read_csv('accepted.csv')
a.head(3)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,246

In [3]:
a.shape

(2260701, 151)

In [4]:
#1. Remove all obserations with No Funded Amount First, How many:
print(len(a[a.funded_amnt.isnull()]))

33


In [5]:
drop = a[a.funded_amnt.isnull()].index
a.drop(drop, axis=0, inplace=True)
a.shape

(2260668, 151)

In [6]:
#No more NaN funded amounts: 
a[a.funded_amnt.isna()].shape

(0, 151)

In [7]:
#2. How many value types for "loan_status"?
print(a.loan_status.value_counts(dropna=False).sum()) #Total matches

a.loan_status.value_counts(dropna=False)

2260668


Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [8]:
#Create new column and map Charged Off = 0, Fully Paid = 1
a['loan_status_bin'] = a['loan_status'].map({'Charged Off':0, 'Fully Paid':1})

In [9]:
#Quick Check:
a[['loan_status', 'loan_status_bin']].sample(10)

,loan_status,loan_status_bin
1429671,Current,NaN
1239761,Fully Paid,1.0
686875,Fully Paid,1.0
84392,Fully Paid,1.0
193532,Fully Paid,1.0
2074391,Current,NaN
1649821,Charged Off,0.0
1099903,Fully Paid,1.0
476857,Current,NaN
67848,Fully Paid,1.0


In [10]:
#Drop all loans which do not classify as "Fully Paid" or "Charged Off":
drop2 = a[a.loan_status_bin.isnull()].index
a.drop(drop2, axis=0, inplace=True)
a.shape

(1345310, 152)

In [11]:
## Keep verification....see what happens: 

#3. Drop all observations where income is not verified: 

a['verification_status'].value_counts()

Source Verified    521273
Verified           418336
Not Verified       405701
Name: verification_status, dtype: int64

In [12]:
#a = a[a['verification_status']!='Not Verified'].reset_index(drop=True)
a.shape

(1345310, 152)

In [13]:
#4. Drop member_id (all NaN), funded_amnt, funded_amnt_inv. Keep loan_amnt (we know that before investment): 
a.drop(['member_id', 'funded_amnt', 'funded_amnt_inv'], axis=1, inplace=True)
a.shape

(1345310, 149)

In [14]:
#5. Remove all columns related to "hardship" status. We won't know this at get go. 

a.drop(['hardship_flag', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount',
        'hardship_loan_status', 'hardship_dpd', 'hardship_end_date', 'hardship_start_date',
        'hardship_amount', 'hardship_status', 'hardship_reason', 'hardship_type', 
        'hardship_length'], axis=1, inplace=True)
a.shape

(1345310, 137)

In [15]:
#6. Remove 'Policy Code' Column. All Policy Codes here are '1'. 
a.policy_code.value_counts()

1.0    1345310
Name: policy_code, dtype: int64

In [16]:
a.drop(['policy_code'], axis=1, inplace=True)
a.shape

(1345310, 136)

In [17]:
#7. Remove all observations with "Joint Applications". 
a.application_type.value_counts()

Individual    1319510
Joint App       25800
Name: application_type, dtype: int64

In [18]:
a = a[a.application_type == 'Individual']
a.shape

(1319510, 136)

In [19]:
#8. Remove all info regarding second applicant or "Joint Applications".

In [20]:
a.drop(['sec_app_mths_since_last_major_derog', 'sec_app_collections_12_mths_ex_med',
        'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts', 'sec_app_open_act_il',
        'sec_app_revol_util', 'sec_app_revol_util', 'sec_app_open_acc', 'sec_app_mort_acc',
        'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line', 'sec_app_fico_range_low',
        'dti_joint', 'sec_app_fico_range_high', 'verification_status_joint', 'revol_bal_joint', 
        'annual_inc_joint'], axis=1, inplace=True)

a.shape

(1319510, 120)

In [21]:
#9. Remove 'next_pymnt_d' because all loans are "Paid Off" or "Defaulted", so they are all "NaN"
a.next_pymnt_d.isna().sum()

1319510

In [22]:
a.drop(['next_pymnt_d'], axis=1, inplace=True)
a.shape

(1319510, 119)

In [23]:
#10. Drop 'url'. It will not provide any info. 
a.drop(['url'], axis=1, inplace=True)
a.shape

(1319510, 118)

In [24]:
#11. Drop 'zip_code'. It's not even complete. Can always use addr_state if location is a factor. 
a.drop(['zip_code'], axis=1, inplace=True)
a.shape

(1319510, 117)

In [25]:
#12. "out_prncp" and "out_prncp_inv" are 0 because there is no more "outstanding" amounts. Drop them. 
print(a.out_prncp.value_counts())
print(a.out_prncp_inv.value_counts())

0.0    1319510
Name: out_prncp, dtype: int64
0.0    1319510
Name: out_prncp_inv, dtype: int64


In [26]:
a.drop(['out_prncp', 'out_prncp_inv'], axis=1, inplace=True)
a.shape

(1319510, 115)

In [27]:
#13. Remove 'id' (irrevelvant) and 'issue_d' (happens after investment committed)
a.drop(['id', 'issue_d'], axis=1, inplace=True)
a.shape

(1319510, 113)

In [28]:
#14. Drop 'pymnt_plan'. They are all 'n'. 
a.pymnt_plan.value_counts()

n    1319510
Name: pymnt_plan, dtype: int64

In [29]:
a.drop(['pymnt_plan'], axis=1, inplace=True)
a.shape

(1319510, 112)

In [30]:
#15. Drop 'last_fico_range_high' and 'last_fico_range_low'. We only rely on initial fico scores. 
a.drop(['last_fico_range_high', 'last_fico_range_low'], axis=1, inplace=True)
a.shape

(1319510, 110)

In [31]:
#16. Taking all loans with interest rate greater than or equal to 20%: 
a = a[a.int_rate >= 20].reset_index(drop=True)
a.shape

(106009, 110)

In [32]:
#Partial Set of Features, Work in Progress......

In [46]:
#16. Let's rearrange order of columns to enhance our view: 
sub = a[['loan_amnt', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high',
   'term', 'int_rate', 'installment', 'initial_list_status',
   'grade', 'sub_grade', 'emp_length', 'home_ownership',
   'earliest_cr_line', 'open_acc', 'total_acc',
   'revol_bal', 'revol_util',
   'inq_last_6mths',
   
   'delinq_2yrs', 'mths_since_last_delinq','acc_now_delinq',
   'collections_12_mths_ex_med','tot_coll_amt', 'tot_cur_bal',
   'pub_rec', 'mths_since_last_record',
   
   
   'purpose', 'title', 'addr_state', 'loan_status', 
   'loan_status_bin', 
   'application_type',
   'verification_status','emp_title',
   'total_pymnt', 'total_pymnt_inv',
   'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
   'recoveries', 'collection_recovery_fee',
   'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 
   'desc']]

sub.head(2)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,pub_rec,mths_since_last_record,purpose,title,addr_state,loan_status,loan_status_bin,application_type,verification_status,emp_title,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,desc
0,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,3 years,MORTGAGE,Jun-1998,12.0,35.0,21929.0,64.5,3.0,1.0,12.0,0.0,0.0,0.0,331730.0,0.0,NaN,major_purchase,Major purchase,PA,Fully Paid,1.0,Individual,Source Verified,Contract Specialist,11740.50,11740.50,10400.00,1340.50,0.0,0.0,0.0,Jul-2016,10128.96,Mar-2018,NaN
1,24250.0,75000.0,20.84,660.0,664.0,60 months,24.24,701.01,w,F,F3,4 years,MORTGAGE,Apr-2007,18.0,21.0,24799.0,65.3,0.0,0.0,NaN,0.0,0.0,0.0,320098.0,0.0,NaN,debt_consolidation,NaN,NY,Charged Off,0.0,Individual,Not Verified,hvac technician,4124.42,4124.42,1332.69,2791.73,0.0,0.0,0.0,Jul-2016,701.01,Mar-2018,NaN


In [47]:
sub.shape

(106009, 46)

In [48]:
sub.isna().mean().sort_values()

loan_amnt                     0.000000
pub_rec                       0.000000
purpose                       0.000000
addr_state                    0.000000
loan_status                   0.000000
loan_status_bin               0.000000
application_type              0.000000
verification_status           0.000000
total_pymnt                   0.000000
total_pymnt_inv               0.000000
total_rec_prncp               0.000000
total_rec_int                 0.000000
total_rec_late_fee            0.000000
recoveries                    0.000000
collection_recovery_fee       0.000000
last_pymnt_amnt               0.000000
acc_now_delinq                0.000000
delinq_2yrs                   0.000000
collections_12_mths_ex_med    0.000000
sub_grade                     0.000000
grade                         0.000000
inq_last_6mths                0.000000
int_rate                      0.000000
home_ownership                0.000000
earliest_cr_line              0.000000
term                     

In [49]:
#Carve out features from 'sub' which you think are most important:

imp_sub = sub[['loan_amnt', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high',
   'term', 'int_rate', 'installment', 'initial_list_status',
   'grade', 'sub_grade', 'emp_length', 'home_ownership',
   'earliest_cr_line', 'open_acc', 'total_acc',
   'revol_bal', 'revol_util',
   'inq_last_6mths', 'verification_status',
   
   'delinq_2yrs', 'mths_since_last_delinq','acc_now_delinq',
   'collections_12_mths_ex_med','tot_coll_amt', 'tot_cur_bal',
   'pub_rec', 'mths_since_last_record',
   
   
   'purpose', 'title', 'loan_status', 
   'loan_status_bin']] 

In [50]:
imp_sub.shape

(106009, 32)

In [51]:
imp_sub.head(2)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,emp_length,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,revol_util,inq_last_6mths,verification_status,delinq_2yrs,mths_since_last_delinq,acc_now_delinq,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,pub_rec,mths_since_last_record,purpose,title,loan_status,loan_status_bin
0,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,3 years,MORTGAGE,Jun-1998,12.0,35.0,21929.0,64.5,3.0,Source Verified,1.0,12.0,0.0,0.0,0.0,331730.0,0.0,NaN,major_purchase,Major purchase,Fully Paid,1.0
1,24250.0,75000.0,20.84,660.0,664.0,60 months,24.24,701.01,w,F,F3,4 years,MORTGAGE,Apr-2007,18.0,21.0,24799.0,65.3,0.0,Not Verified,0.0,NaN,0.0,0.0,0.0,320098.0,0.0,NaN,debt_consolidation,NaN,Charged Off,0.0


In [52]:
imp_sub.isna().sum().sort_values()

loan_amnt                         0
purpose                           0
pub_rec                           0
collections_12_mths_ex_med        0
acc_now_delinq                    0
delinq_2yrs                       0
verification_status               0
inq_last_6mths                    0
revol_bal                         0
loan_status                       0
open_acc                          0
earliest_cr_line                  0
total_acc                         0
fico_range_high                   0
annual_inc                        0
dti                               0
fico_range_low                    0
home_ownership                    0
term                              0
loan_status_bin                   0
installment                       0
initial_list_status               0
grade                             0
sub_grade                         0
int_rate                          0
revol_util                      111
title                          1666
tot_coll_amt                

In [53]:
#For the time being, drop all features with NAs....we don't know which ones are important.
#We also don't know what is the best way to "impute".
No_NA = imp_sub.dropna(axis=1)
No_NA

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin
0,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,MORTGAGE,Jun-1998,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0
1,24250.0,75000.0,20.84,660.0,664.0,60 months,24.24,701.01,w,F,F3,MORTGAGE,Apr-2007,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0
2,15850.0,45000.0,34.85,755.0,759.0,60 months,23.13,448.01,f,F,F2,OWN,Jan-2007,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
3,23100.0,110000.0,20.43,660.0,664.0,60 months,20.50,618.46,w,E,E4,OWN,Aug-1986,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0
4,35000.0,120000.0,35.69,700.0,704.0,60 months,20.50,937.06,w,E,E4,MORTGAGE,Oct-1994,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,60 months,21.49,956.55,f,D,D5,RENT,Oct-2006,5.0,16.0,16267.0,0.0,Verified,0.0,0.0,0.0,0.0,major_purchase,Charged Off,0.0
106005,15650.0,58000.0,27.81,685.0,689.0,60 months,26.49,473.12,w,F,F2,MORTGAGE,Jul-2002,13.0,29.0,8273.0,1.0,Verified,0.0,0.0,0.0,1.0,debt_consolidation,Fully Paid,1.0
106006,27450.0,80000.0,15.17,705.0,709.0,36 months,21.49,1041.10,f,D,D5,MORTGAGE,Jun-2007,12.0,21.0,6220.0,0.0,Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0
106007,20000.0,75000.0,25.23,670.0,674.0,60 months,21.49,546.60,f,D,D5,MORTGAGE,Jul-1998,9.0,21.0,19185.0,1.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0


In [54]:
#We start with 24 features (includes target as well)
No_NA.shape

(106009, 25)

In [55]:
#Cross check that dataframe in use has no more Null values:
No_NA.isna().sum().sum()

0

In [56]:
data = No_NA.copy()

In [57]:
#Data Types of Remaining Features: 
data.dtypes

loan_amnt                     float64
annual_inc                    float64
dti                           float64
fico_range_low                float64
fico_range_high               float64
term                           object
int_rate                      float64
installment                   float64
initial_list_status            object
grade                          object
sub_grade                      object
home_ownership                 object
earliest_cr_line               object
open_acc                      float64
total_acc                     float64
revol_bal                     float64
inq_last_6mths                float64
verification_status            object
delinq_2yrs                   float64
acc_now_delinq                float64
collections_12_mths_ex_med    float64
pub_rec                       float64
purpose                        object
loan_status                    object
loan_status_bin               float64
dtype: object

In [58]:
#Return unique values of categorical columns:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column]=='object'}

{'term': [' 60 months', ' 36 months'],
 'initial_list_status': ['w', 'f'],
 'grade': ['F', 'E', 'G', 'D'],
 'sub_grade': ['F1',
  'F3',
  'F2',
  'E4',
  'F5',
  'E5',
  'G2',
  'G1',
  'G3',
  'G4',
  'F4',
  'G5',
  'D5',
  'D4',
  'E2',
  'E3',
  'E1'],
 'home_ownership': ['MORTGAGE', 'OWN', 'RENT', 'ANY', 'NONE', 'OTHER'],
 'earliest_cr_line': ['Jun-1998',
  'Apr-2007',
  'Jan-2007',
  'Aug-1986',
  'Oct-1994',
  'Jan-2002',
  'Nov-1996',
  'Jun-1997',
  'Feb-1998',
  'Oct-2001',
  'Sep-2006',
  'Apr-1990',
  'Oct-1999',
  'Mar-2002',
  'Jul-2008',
  'May-1981',
  'Jul-1999',
  'Jan-1992',
  'Nov-2007',
  'Nov-1999',
  'Apr-2008',
  'Jan-2001',
  'Jun-1993',
  'May-1998',
  'Jan-1996',
  'Dec-2002',
  'Oct-2004',
  'Oct-2011',
  'Dec-1987',
  'Mar-2006',
  'Dec-1990',
  'Mar-2007',
  'Dec-2008',
  'Feb-1993',
  'Aug-1996',
  'Jun-1990',
  'Sep-1994',
  'Sep-1989',
  'Jul-2003',
  'Jul-1988',
  'Jan-2006',
  'May-1996',
  'Nov-1986',
  'Aug-2002',
  'Jan-1999',
  'Jan-1987',
  'May-

In [59]:
#A. Working with Date Columns First: 

In [60]:
date_columns = ['earliest_cr_line']

In [61]:
data[date_columns]

,earliest_cr_line
0,Jun-1998
1,Apr-2007
2,Jan-2007
3,Aug-1986
4,Oct-1994
...,...
106004,Oct-2006
106005,Jul-2002
106006,Jun-2007
106007,Jul-1998


In [62]:
data.loc[0, 'earliest_cr_line'][0:3]

'Jun'

In [63]:
data.loc[0, 'earliest_cr_line'][-4:]

'1998'

In [64]:
for column in date_columns:
    data[column + '_month'] = data[column].apply(lambda x: x[0:3])
    data[column + '_year'] = data[column].apply(lambda x: x[-4:])

In [65]:
data

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,home_ownership,earliest_cr_line,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year
0,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,MORTGAGE,Jun-1998,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,Jun,1998
1,24250.0,75000.0,20.84,660.0,664.0,60 months,24.24,701.01,w,F,F3,MORTGAGE,Apr-2007,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,Apr,2007
2,15850.0,45000.0,34.85,755.0,759.0,60 months,23.13,448.01,f,F,F2,OWN,Jan-2007,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Jan,2007
3,23100.0,110000.0,20.43,660.0,664.0,60 months,20.50,618.46,w,E,E4,OWN,Aug-1986,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,Aug,1986
4,35000.0,120000.0,35.69,700.0,704.0,60 months,20.50,937.06,w,E,E4,MORTGAGE,Oct-1994,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Oct,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,60 months,21.49,956.55,f,D,D5,RENT,Oct-2006,5.0,16.0,16267.0,0.0,Verified,0.0,0.0,0.0,0.0,major_purchase,Charged Off,0.0,Oct,2006
106005,15650.0,58000.0,27.81,685.0,689.0,60 months,26.49,473.12,w,F,F2,MORTGAGE,Jul-2002,13.0,29.0,8273.0,1.0,Verified,0.0,0.0,0.0,1.0,debt_consolidation,Fully Paid,1.0,Jul,2002
106006,27450.0,80000.0,15.17,705.0,709.0,36 months,21.49,1041.10,f,D,D5,MORTGAGE,Jun-2007,12.0,21.0,6220.0,0.0,Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Jun,2007
106007,20000.0,75000.0,25.23,670.0,674.0,60 months,21.49,546.60,f,D,D5,MORTGAGE,Jul-1998,9.0,21.0,19185.0,1.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,Jul,1998


In [66]:
data = data.drop(date_columns, axis=1)
data.shape

(106009, 26)

In [67]:
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [68]:
for column in date_columns: 
    data[column + '_month'] = data[column + '_month'].apply(lambda x: month_ordering.index(x))

In [69]:
data

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,term,int_rate,installment,initial_list_status,grade,sub_grade,home_ownership,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year
0,10400.0,104433.0,25.37,695.0,699.0,60 months,22.45,289.91,w,F,F1,MORTGAGE,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5,1998
1,24250.0,75000.0,20.84,660.0,664.0,60 months,24.24,701.01,w,F,F3,MORTGAGE,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3,2007
2,15850.0,45000.0,34.85,755.0,759.0,60 months,23.13,448.01,f,F,F2,OWN,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0,2007
3,23100.0,110000.0,20.43,660.0,664.0,60 months,20.50,618.46,w,E,E4,OWN,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7,1986
4,35000.0,120000.0,35.69,700.0,704.0,60 months,20.50,937.06,w,E,E4,MORTGAGE,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,60 months,21.49,956.55,f,D,D5,RENT,5.0,16.0,16267.0,0.0,Verified,0.0,0.0,0.0,0.0,major_purchase,Charged Off,0.0,9,2006
106005,15650.0,58000.0,27.81,685.0,689.0,60 months,26.49,473.12,w,F,F2,MORTGAGE,13.0,29.0,8273.0,1.0,Verified,0.0,0.0,0.0,1.0,debt_consolidation,Fully Paid,1.0,6,2002
106006,27450.0,80000.0,15.17,705.0,709.0,36 months,21.49,1041.10,f,D,D5,MORTGAGE,12.0,21.0,6220.0,0.0,Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,5,2007
106007,20000.0,75000.0,25.23,670.0,674.0,60 months,21.49,546.60,f,D,D5,MORTGAGE,9.0,21.0,19185.0,1.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,6,1998


In [70]:
for column in data.columns:
    try:
        data[column] = data[column].astype(np.float)
    except:
        pass

In [71]:
data.dtypes

loan_amnt                     float64
annual_inc                    float64
dti                           float64
fico_range_low                float64
fico_range_high               float64
term                           object
int_rate                      float64
installment                   float64
initial_list_status            object
grade                          object
sub_grade                      object
home_ownership                 object
open_acc                      float64
total_acc                     float64
revol_bal                     float64
inq_last_6mths                float64
verification_status            object
delinq_2yrs                   float64
acc_now_delinq                float64
collections_12_mths_ex_med    float64
pub_rec                       float64
purpose                        object
loan_status                    object
loan_status_bin               float64
earliest_cr_line_month        float64
earliest_cr_line_year         float64
dtype: objec

In [72]:
#Return unique values of categorical columns:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column]=='object'}

{'term': [' 60 months', ' 36 months'],
 'initial_list_status': ['w', 'f'],
 'grade': ['F', 'E', 'G', 'D'],
 'sub_grade': ['F1',
  'F3',
  'F2',
  'E4',
  'F5',
  'E5',
  'G2',
  'G1',
  'G3',
  'G4',
  'F4',
  'G5',
  'D5',
  'D4',
  'E2',
  'E3',
  'E1'],
 'home_ownership': ['MORTGAGE', 'OWN', 'RENT', 'ANY', 'NONE', 'OTHER'],
 'verification_status': ['Source Verified', 'Not Verified', 'Verified'],
 'purpose': ['major_purchase',
  'debt_consolidation',
  'car',
  'other',
  'credit_card',
  'home_improvement',
  'medical',
  'vacation',
  'small_business',
  'moving',
  'house',
  'renewable_energy',
  'wedding',
  'educational'],
 'loan_status': ['Fully Paid', 'Charged Off']}

In [73]:
#Encoding Functions

def binary_temp(df, column, positive_value):
    temp_df = df.assign(new_be=df[column].apply(lambda x: 1 if x == positive_value else 0))
    return temp_df

def binary_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_be':column})
    return new_df
    
    #df[column + '_be'] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    #df_new = pd.concat([df, df[column].apply(lambda x: 1 if x == positive_value else 0)], axis=1)
    #df_new = df_new.drop(column, axis=1)
    #return df_new

def ordinal_temp(df, column, ordering):
    temp_df = df.assign(new_oe=df[column].apply(lambda x: ordering.index(x)))
    return temp_df

def ordinal_encode(temp_df, column):
    new_df = temp_df.drop(column, axis=1)
    new_df = new_df.rename(columns={'new_oe':column})
    return new_df

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new

In [74]:
#Check Binary Encoding:
temp = binary_temp(data, 'term', ' 60 months')

In [75]:
temp[['term', 'new_be']].sample(10)

,term,new_be
93941,60 months,1
63021,60 months,1
88259,60 months,1
86483,60 months,1
98988,60 months,1
39877,60 months,1
86000,36 months,0
92287,36 months,0
82850,60 months,1
81466,60 months,1


In [76]:
#If ok, complete binary encoding:
new_df = binary_encode(temp, 'term')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,initial_list_status,grade,sub_grade,home_ownership,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,w,F,F1,MORTGAGE,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5.0,1998.0,1
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,w,F,F3,MORTGAGE,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3.0,2007.0,1
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,f,F,F2,OWN,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0.0,2007.0,1
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,w,E,E4,OWN,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7.0,1986.0,1
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,w,E,E4,MORTGAGE,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9.0,1994.0,1


In [77]:
temp = binary_temp(new_df, 'initial_list_status', 'w')

In [78]:
temp[['initial_list_status', 'new_be']].sample(10)

,initial_list_status,new_be
27095,f,0
90404,w,1
54826,w,1
104882,f,0
37806,w,1
34806,w,1
88723,f,0
96699,w,1
64831,f,0
34938,f,0


In [79]:
new_df = binary_encode(temp, 'initial_list_status')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,grade,sub_grade,home_ownership,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,F,F1,MORTGAGE,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5.0,1998.0,1,1
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,F,F3,MORTGAGE,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3.0,2007.0,1,1
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,F,F2,OWN,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0.0,2007.0,1,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,E,E4,OWN,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7.0,1986.0,1,1
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,E,E4,MORTGAGE,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9.0,1994.0,1,1


In [80]:
grade_ordering = sorted(new_df['grade'].unique())
grade_ordering

['D', 'E', 'F', 'G']

In [81]:
temp_df = ordinal_temp(new_df, 'grade', grade_ordering)
temp_df[['grade', 'new_oe']]

,grade,new_oe
0,F,2
1,F,2
2,F,2
3,E,1
4,E,1
...,...,...
106004,D,0
106005,F,2
106006,D,0
106007,D,0


In [82]:
#If ok, complete ordinal encoding:
new_df = ordinal_encode(temp_df, 'grade')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,sub_grade,home_ownership,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,F1,MORTGAGE,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5.0,1998.0,1,1,2
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,F3,MORTGAGE,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3.0,2007.0,1,1,2
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,F2,OWN,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0.0,2007.0,1,0,2
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,E4,OWN,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7.0,1986.0,1,1,1
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,E4,MORTGAGE,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9.0,1994.0,1,1,1


In [83]:
sub_grade_ordering = sorted(new_df['sub_grade'].unique())
sub_grade_ordering

['D4',
 'D5',
 'E1',
 'E2',
 'E3',
 'E4',
 'E5',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'G1',
 'G2',
 'G3',
 'G4',
 'G5']

In [84]:
temp_df = ordinal_temp(new_df, 'sub_grade', sub_grade_ordering)
temp_df[['sub_grade', 'new_oe']]

,sub_grade,new_oe
0,F1,7
1,F3,9
2,F2,8
3,E4,5
4,E4,5
...,...,...
106004,D5,1
106005,F2,8
106006,D5,1
106007,D5,1


In [85]:
#If ok, complete ordinal encoding:
new_df = ordinal_encode(temp_df, 'sub_grade')
new_df.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,home_ownership,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,MORTGAGE,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5.0,1998.0,1,1,2,7
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,MORTGAGE,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3.0,2007.0,1,1,2,9
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,OWN,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0.0,2007.0,1,0,2,8
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,OWN,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7.0,1986.0,1,1,1,5
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,MORTGAGE,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9.0,1994.0,1,1,1,5


In [86]:
#Dummify nominal features:
numerical_df = onehot_encode(new_df, 'home_ownership')
numerical_df

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,purpose,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,major_purchase,Fully Paid,1.0,5.0,1998.0,1,1,2,7,0,1,0,0,0,0
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Charged Off,0.0,3.0,2007.0,1,1,2,9,0,1,0,0,0,0
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,0.0,2007.0,1,0,2,8,0,0,0,0,1,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,car,Charged Off,0.0,7.0,1986.0,1,1,1,5,0,0,0,0,1,0
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,9.0,1994.0,1,1,1,5,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,21.49,956.55,5.0,16.0,16267.0,0.0,Verified,0.0,0.0,0.0,0.0,major_purchase,Charged Off,0.0,9.0,2006.0,1,0,0,1,0,0,0,0,0,1
106005,15650.0,58000.0,27.81,685.0,689.0,26.49,473.12,13.0,29.0,8273.0,1.0,Verified,0.0,0.0,0.0,1.0,debt_consolidation,Fully Paid,1.0,6.0,2002.0,1,1,2,8,0,1,0,0,0,0
106006,27450.0,80000.0,15.17,705.0,709.0,21.49,1041.10,12.0,21.0,6220.0,0.0,Verified,0.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,5.0,2007.0,0,0,0,1,0,1,0,0,0,0
106007,20000.0,75000.0,25.23,670.0,674.0,21.49,546.60,9.0,21.0,19185.0,1.0,Source Verified,1.0,0.0,0.0,0.0,debt_consolidation,Fully Paid,1.0,6.0,1998.0,1,0,0,1,0,1,0,0,0,0


In [87]:
numerical_df2 = onehot_encode(numerical_df, 'purpose')
numerical_df2

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,verification_status,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,3.0,Source Verified,1.0,0.0,0.0,0.0,Fully Paid,1.0,5.0,1998.0,1,1,2,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,18.0,21.0,24799.0,0.0,Not Verified,0.0,0.0,0.0,0.0,Charged Off,0.0,3.0,2007.0,1,1,2,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,18.0,24.0,8799.0,1.0,Not Verified,0.0,0.0,0.0,0.0,Fully Paid,1.0,0.0,2007.0,1,0,2,8,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,11.0,24.0,20594.0,0.0,Not Verified,0.0,0.0,0.0,0.0,Charged Off,0.0,7.0,1986.0,1,1,1,5,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,25.0,54.0,13993.0,0.0,Source Verified,1.0,0.0,0.0,0.0,Fully Paid,1.0,9.0,1994.0,1,1,1,5,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,21.49,956.55,5.0,16.0,16267.0,0.0,Verified,0.0,0.0,0.0,0.0,Charged Off,0.0,9.0,2006.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
106005,15650.0,58000.0,27.81,685.0,689.0,26.49,473.12,13.0,29.0,8273.0,1.0,Verified,0.0,0.0,0.0,1.0,Fully Paid,1.0,6.0,2002.0,1,1,2,8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
106006,27450.0,80000.0,15.17,705.0,709.0,21.49,1041.10,12.0,21.0,6220.0,0.0,Verified,0.0,0.0,0.0,0.0,Fully Paid,1.0,5.0,2007.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
106007,20000.0,75000.0,25.23,670.0,674.0,21.49,546.60,9.0,21.0,19185.0,1.0,Source Verified,1.0,0.0,0.0,0.0,Fully Paid,1.0,6.0,1998.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [88]:
numerical_df3 = onehot_encode(numerical_df2, 'verification_status')
numerical_df3

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,loan_status,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,3.0,1.0,0.0,0.0,0.0,Fully Paid,1.0,5.0,1998.0,1,1,2,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,18.0,21.0,24799.0,0.0,0.0,0.0,0.0,0.0,Charged Off,0.0,3.0,2007.0,1,1,2,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,18.0,24.0,8799.0,1.0,0.0,0.0,0.0,0.0,Fully Paid,1.0,0.0,2007.0,1,0,2,8,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,11.0,24.0,20594.0,0.0,0.0,0.0,0.0,0.0,Charged Off,0.0,7.0,1986.0,1,1,1,5,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,25.0,54.0,13993.0,0.0,1.0,0.0,0.0,0.0,Fully Paid,1.0,9.0,1994.0,1,1,1,5,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,21.49,956.55,5.0,16.0,16267.0,0.0,0.0,0.0,0.0,0.0,Charged Off,0.0,9.0,2006.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
106005,15650.0,58000.0,27.81,685.0,689.0,26.49,473.12,13.0,29.0,8273.0,1.0,0.0,0.0,0.0,1.0,Fully Paid,1.0,6.0,2002.0,1,1,2,8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106006,27450.0,80000.0,15.17,705.0,709.0,21.49,1041.10,12.0,21.0,6220.0,0.0,0.0,0.0,0.0,0.0,Fully Paid,1.0,5.0,2007.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106007,20000.0,75000.0,25.23,670.0,674.0,21.49,546.60,9.0,21.0,19185.0,1.0,1.0,0.0,0.0,0.0,Fully Paid,1.0,6.0,1998.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [89]:
numerical_df4 = numerical_df3.drop('loan_status', axis=1)
numerical_df4

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,loan_status_bin,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,3.0,1.0,0.0,0.0,0.0,1.0,5.0,1998.0,1,1,2,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,18.0,21.0,24799.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2007.0,1,1,2,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,18.0,24.0,8799.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2007.0,1,0,2,8,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,11.0,24.0,20594.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1986.0,1,1,1,5,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,25.0,54.0,13993.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,1994.0,1,1,1,5,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,21.49,956.55,5.0,16.0,16267.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
106005,15650.0,58000.0,27.81,685.0,689.0,26.49,473.12,13.0,29.0,8273.0,1.0,0.0,0.0,0.0,1.0,1.0,6.0,2002.0,1,1,2,8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106006,27450.0,80000.0,15.17,705.0,709.0,21.49,1041.10,12.0,21.0,6220.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2007.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106007,20000.0,75000.0,25.23,670.0,674.0,21.49,546.60,9.0,21.0,19185.0,1.0,1.0,0.0,0.0,0.0,1.0,6.0,1998.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [90]:
numerical_df4.dtypes

loan_amnt                     float64
annual_inc                    float64
dti                           float64
fico_range_low                float64
fico_range_high               float64
int_rate                      float64
installment                   float64
open_acc                      float64
total_acc                     float64
revol_bal                     float64
inq_last_6mths                float64
delinq_2yrs                   float64
acc_now_delinq                float64
collections_12_mths_ex_med    float64
pub_rec                       float64
loan_status_bin               float64
earliest_cr_line_month        float64
earliest_cr_line_year         float64
term                            int64
initial_list_status             int64
grade                           int64
sub_grade                       int64
ANY                             uint8
MORTGAGE                        uint8
NONE                            uint8
OTHER                           uint8
OWN         

In [91]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [97]:
X = numerical_df4.drop('loan_status_bin', axis=1)
y = numerical_df4['loan_status_bin']

In [98]:
X

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,10400.0,104433.0,25.37,695.0,699.0,22.45,289.91,12.0,35.0,21929.0,3.0,1.0,0.0,0.0,0.0,5.0,1998.0,1,1,2,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,24250.0,75000.0,20.84,660.0,664.0,24.24,701.01,18.0,21.0,24799.0,0.0,0.0,0.0,0.0,0.0,3.0,2007.0,1,1,2,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,15850.0,45000.0,34.85,755.0,759.0,23.13,448.01,18.0,24.0,8799.0,1.0,0.0,0.0,0.0,0.0,0.0,2007.0,1,0,2,8,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,23100.0,110000.0,20.43,660.0,664.0,20.50,618.46,11.0,24.0,20594.0,0.0,0.0,0.0,0.0,0.0,7.0,1986.0,1,1,1,5,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,35000.0,120000.0,35.69,700.0,704.0,20.50,937.06,25.0,54.0,13993.0,0.0,1.0,0.0,0.0,0.0,9.0,1994.0,1,1,1,5,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,35000.0,72000.0,16.63,675.0,679.0,21.49,956.55,5.0,16.0,16267.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
106005,15650.0,58000.0,27.81,685.0,689.0,26.49,473.12,13.0,29.0,8273.0,1.0,0.0,0.0,0.0,1.0,6.0,2002.0,1,1,2,8,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106006,27450.0,80000.0,15.17,705.0,709.0,21.49,1041.10,12.0,21.0,6220.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106007,20000.0,75000.0,25.23,670.0,674.0,21.49,546.60,9.0,21.0,19185.0,1.0,1.0,0.0,0.0,0.0,6.0,1998.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [99]:
y

0         1.0
1         0.0
2         1.0
3         0.0
4         1.0
         ... 
106004    0.0
106005    1.0
106006    1.0
106007    1.0
106008    1.0
Name: loan_status_bin, Length: 106009, dtype: float64

In [100]:
X_scaled = sc.fit_transform(X)

In [101]:
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,-0.805788,0.544423,0.570300,0.663197,0.663194,-0.388527,-0.928141,0.020404,0.814165,0.277188,1.683433,0.681092,-0.074467,-0.125292,-0.377704,-0.220266,-0.283574,0.729709,1.019288,0.837092,0.319365,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,6.888409,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,1.298703,-1.011585
1,0.650069,0.015948,0.055254,-1.040497,-1.040493,0.282002,0.487904,1.071777,-0.307337,0.416038,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,-0.808202,0.974220,0.729709,1.019288,0.837092,0.871554,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585
2,-0.232905,-0.522707,1.648144,3.583815,3.583802,-0.133801,-0.383561,1.071777,-0.067015,-0.358038,-0.040284,-0.383738,-0.074467,-0.125292,-0.377704,-1.690107,0.974220,0.729709,-0.981077,0.837092,0.595459,-0.017646,-0.909699,-0.007523,-0.007523,2.788603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585
3,0.529186,0.644379,0.008638,-1.040497,-1.040493,-1.118991,0.203558,-0.154825,-0.067015,0.212601,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,0.367670,-1.960634,0.729709,1.019288,-0.426533,-0.232824,-0.017646,-0.909699,-0.007523,-0.007523,2.788603,-0.873547,11.902814,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585
4,1.780067,0.823931,1.743649,0.906582,0.906578,-1.118991,1.300984,2.298379,2.336203,-0.106754,-0.902143,0.681092,-0.074467,-0.125292,-0.377704,0.955607,-0.842594,0.729709,1.019288,-0.426533,-0.232824,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,1.298703,-1.011585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,1.780067,-0.037917,-0.423409,-0.310342,-0.310341,-0.748140,1.368118,-1.206197,-0.707874,0.003262,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,0.955607,0.834465,0.729709,-0.981077,-1.690158,-1.337202,-0.017646,-0.909699,-0.007523,-0.007523,-0.358603,1.144758,-0.084014,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,6.888409,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,-0.769999,0.988548
106005,-0.253929,-0.289290,0.847719,0.176428,0.176426,1.124845,-0.297069,0.195633,0.333521,-0.383486,-0.040284,-0.383738,-0.074467,-0.125292,1.199856,0.073702,0.275446,0.729709,1.019288,0.837092,0.595459,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,-0.769999,0.988548
106006,0.986441,0.105724,-0.589406,1.149967,1.149962,-0.748140,1.659352,0.020404,-0.307337,-0.482810,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,-0.220266,0.974220,-1.370409,-0.981077,-1.690158,-1.337202,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.2

In [102]:
scaled = pd.concat([X,y], axis=1)
scaled

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,pub_rec,earliest_cr_line_month,earliest_cr_line_year,term,initial_list_status,grade,sub_grade,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified,loan_status_bin
0,-0.805788,0.544423,0.570300,0.663197,0.663194,-0.388527,-0.928141,0.020404,0.814165,0.277188,1.683433,0.681092,-0.074467,-0.125292,-0.377704,-0.220266,-0.283574,0.729709,1.019288,0.837092,0.319365,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,6.888409,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,1.298703,-1.011585,1.0
1,0.650069,0.015948,0.055254,-1.040497,-1.040493,0.282002,0.487904,1.071777,-0.307337,0.416038,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,-0.808202,0.974220,0.729709,1.019288,0.837092,0.871554,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585,0.0
2,-0.232905,-0.522707,1.648144,3.583815,3.583802,-0.133801,-0.383561,1.071777,-0.067015,-0.358038,-0.040284,-0.383738,-0.074467,-0.125292,-0.377704,-1.690107,0.974220,0.729709,-0.981077,0.837092,0.595459,-0.017646,-0.909699,-0.007523,-0.007523,2.788603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585,1.0
3,0.529186,0.644379,0.008638,-1.040497,-1.040493,-1.118991,0.203558,-0.154825,-0.067015,0.212601,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,0.367670,-1.960634,0.729709,1.019288,-0.426533,-0.232824,-0.017646,-0.909699,-0.007523,-0.007523,2.788603,-0.873547,11.902814,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,2.682328,-0.769999,-1.011585,0.0
4,1.780067,0.823931,1.743649,0.906582,0.906578,-1.118991,1.300984,2.298379,2.336203,-0.106754,-0.902143,0.681092,-0.074467,-0.125292,-0.377704,0.955607,-0.842594,0.729709,1.019288,-0.426533,-0.232824,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,1.298703,-1.011585,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106004,1.780067,-0.037917,-0.423409,-0.310342,-0.310341,-0.748140,1.368118,-1.206197,-0.707874,0.003262,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,0.955607,0.834465,0.729709,-0.981077,-1.690158,-1.337202,-0.017646,-0.909699,-0.007523,-0.007523,-0.358603,1.144758,-0.084014,-0.342123,-1.311086,-0.003071,-0.252444,-0.117473,6.888409,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,-0.769999,0.988548,0.0
106005,-0.253929,-0.289290,0.847719,0.176428,0.176426,1.124845,-0.297069,0.195633,0.333521,-0.383486,-0.040284,-0.383738,-0.074467,-0.125292,1.199856,0.073702,0.275446,0.729709,1.019288,0.837092,0.595459,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.873547,-0.084014,-0.342123,0.762727,-0.003071,-0.252444,-0.117473,-0.145171,-0.123241,-0.114845,-0.323001,-0.037391,-0.171212,-0.082752,-0.052011,-0.372811,-0.769999,0.988548,1.0
106006,0.986441,0.105724,-0.589406,1.149967,1.149962,-0.748140,1.659352,0.020404,-0.307337,-0.482810,-0.902143,-0.383738,-0.074467,-0.125292,-0.377704,-0.220266,0.974220,-1.370409,-0.981077,-1.690158,-1.337202,-0.017646,1.099265,-0.007523,-0.007523,-0.358603,-0.87

In [103]:
scaled.to_csv('20x45.csv', index=False)

In [ ]:
##Continue from here? Go back to Atkin video to see how he cleans up data. 
##Combine with your procedures below.....

In [ ]:
#Below are possible ideas/ways to impute or 
#include features with missing values, if deemed important: 

In [ ]:
# 1.Emp Length: 
#below is to show why we categorize "NaN" for emp_length under "0" by pay off ratio. 

In [ ]:
a.head(2)

In [ ]:
a[a.emp_length.isna()].loan_status_bin.mean()

In [ ]:
sns.countplot(data=a, y='emp_length', hue='loan_status')

In [ ]:
a[['emp_length', 'loan_status_bin']].groupby('emp_length').mean()

In [ ]:
#2. Remove "Debt Settlement" Related Features?? These come into play after loan has been "Charged Off". 

In [ ]:
a.debt_settlement_flag.value_counts()

In [ ]:
a[a.debt_settlement_flag == "Y"]['loan_status'].value_counts()

In [ ]:
a[(a.debt_settlement_flag == 'Y') & (a.loan_status=='Fully Paid')]

In [ ]:
#Below is checking original dataframe, its shape, its features and definitions: 

In [ ]:
a.shape

In [ ]:
a.head(2)

In [ ]:
a.isna().mean().sort_values()

In [ ]:
#Data Dictionary to review features: 
pd.read_excel('LCDataDictionary.xlsx')

In [ ]:
new_df['emp_length'].replace(np.nan, "None", inplace=True)

In [ ]:
new_df['emp_length'].value_counts()

In [ ]:
emp_ordering = [
    'None',
    '< 1 year',
    '1 year',
    '2 years',
    '3 years',
    '4 years',
    '5 years',
    '6 years',
    '7 years',
    '8 years',
    '9 years',
    '10+ years',
    ]

In [ ]:
temp_df = ordinal_temp(new_df, 'emp_length', emp_ordering)
temp_df[['emp_length', 'new_oe']]

In [ ]:
new_df = ordinal_encode(temp_df, 'emp_length')
new_df.head(3)

In [ ]:
#All features are now in numerical form
numerical_df.dtypes

In [ ]:
#Check if any missing values left, especially in numerical columns which were not transformed:
numerical_df.isna().sum()  #70 values in dti. Let's drop them. 

In [ ]:
len(numerical_df[numerical_df.dti.isna()])

In [ ]:
len(numerical_df[numerical_df.annual_inc==0])

In [ ]:
#Drop these 5 observations for now, may revisit later....

numerical_df = numerical_df.dropna().reset_index(drop=True)
numerical_df

In [ ]:
#numerical_df.to_csv('20%_Loans_12_feat_unscaled_verified.csv', index=False)